**MODEL CREATION**

In [26]:
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.layers import Input
import tensorflow as tf
from keras import layers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau
from keras.optimizers import SGD, RMSprop

In [27]:
IMG_SIZE = 400
no_of_fruits = 10

In [28]:
def create_model():
	model = tf.keras.models.Sequential()
	model.add(keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3)))
	model.add(layers.Conv2D(32,(5,5),activation='relu',padding='same'))
	model.add(layers.BatchNormalization())
	model.add(layers.Dropout(0.25))
	model.add(layers.MaxPooling2D((5, 5), padding='same'))
	model.add(layers.Conv2D(64,(5,5),activation='relu',padding='same'))
	model.add(layers.BatchNormalization())
	model.add(layers.MaxPooling2D((5, 5), padding='same'))
	model.add(layers.Dropout(0.25))
	model.add(layers.Conv2D(128,(5, 5), activation='relu',padding='same'))
	model.add(layers.BatchNormalization())
	model.add(layers.MaxPooling2D((5, 5), padding='same'))
	model.add(layers.Conv2D(256,(5, 5), activation='relu',padding='same'))
	model.add(layers.BatchNormalization())
	model.add(layers.MaxPooling2D((5, 5), padding='same'))
	model.add(layers.Conv2D(512,(5,5),activation='relu',padding='same'))
	model.add(layers.MaxPooling2D((5, 5), padding='same'))
	model.add(layers.Conv2D(1024,(5,5),activation='relu',padding='same'))
	model.add(layers.MaxPooling2D((5, 5), padding='same'))
	model.add(layers.GlobalAveragePooling2D())
	model.add(layers.Dense(256, activation='relu'))
	model.add(layers.Dense(128, activation='relu'))
	model.add(layers.Dropout(.2))
	model.add(layers.Dense(64, activation='relu'))
	model.add(layers.Dropout(.2))
	model.add(layers.Dense(32, activation='relu'))
	model.add(layers.Dense(no_of_fruits, activation='softmax'))
	return model

**SEGMENT IMAGE**

In [29]:
import cv2
import numpy as np
import os

In [30]:
def calculateFoodArea(image):
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_filter = cv2.medianBlur( img, 5)
    image_threshold = cv2.adaptiveThreshold(image_filter,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,21,2)
    contours, hierarchy = cv2.findContours(image_threshold, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.zeros(img.shape, np.uint8)
    largest_areas = sorted(contours, key=cv2.contourArea)
    cv2.drawContours(mask, [largest_areas[-1]], 0, (255,255,255,255), -1)
    image_contour = cv2.bitwise_and(image,image,mask = mask)
    hsv_image = cv2.cvtColor(image_contour, cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(hsv_image)
    mask_plate = cv2.inRange(hsv_image, np.array([0,0,50]), np.array([200,90,250]))
    mask_not_plate = cv2.bitwise_not(mask_plate)
    fruit_skin = cv2.bitwise_and(image_contour,image_contour,mask = mask_not_plate)
    hsv_image = cv2.cvtColor(fruit_skin, cv2.COLOR_BGR2HSV)
    skin = cv2.inRange(hsv_image, np.array([0,10,60]), np.array([10,160,255]))
    not_skin = cv2.bitwise_not(skin);
    fruit = cv2.bitwise_and(fruit_skin,fruit_skin,mask = not_skin) 
    fruit_black_white = cv2.cvtColor(fruit, cv2.COLOR_BGR2GRAY)
    fruit_bin = cv2.inRange(fruit_black_white, 10, 255) 
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    erode_fruit = cv2.erode(fruit_bin,kernel,iterations = 1)
    image_threshold = cv2.adaptiveThreshold(erode_fruit,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    contours, hierarchy = cv2.findContours(image_threshold, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    mask_fruit = np.zeros(fruit_bin.shape, np.uint8)
    largest_areas = sorted(contours, key=cv2.contourArea)
    cv2.drawContours(mask_fruit, [largest_areas[-2]], 0, (255,255,255), -1)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    mask_fruit = cv2.dilate(mask_fruit,kernel,iterations = 1)
    fruit_final = cv2.bitwise_and(image,image,mask = mask_fruit)
    image_threshold = cv2.adaptiveThreshold(mask_fruit,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    contours, hierarchy = cv2.findContours(image_threshold, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    largest_areas = sorted(contours, key=cv2.contourArea)
    fruit_contour = largest_areas[-2]
    fruit_area = cv2.contourArea(fruit_contour)
    skin2 = skin - mask_fruit
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    skin_e = cv2.erode(skin2,kernel,iterations = 1)
    image_threshold = cv2.adaptiveThreshold(skin_e,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    contours, hierarchy = cv2.findContours(image_threshold, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    mask_skin = np.zeros(skin.shape, np.uint8)
    largest_areas = sorted(contours, key=cv2.contourArea)
    cv2.drawContours(mask_skin, [largest_areas[-2]], 0, (255,255,255), -1)
    skin_rect = cv2.minAreaRect(largest_areas[-2])
    skin_skin_box = cv2.boxPoints(skin_rect)
    skin_skin_box = np.int0(skin_skin_box)
    mask_skin2 = np.zeros(skin.shape, np.uint8)
    cv2.drawContours(mask_skin2,[skin_skin_box],0,(255,255,255), -1)
    pix_height = max(skin_rect[1])
    pix_to_cm_multiplier = 5.0/pix_height
    skin_area = cv2.contourArea(skin_skin_box)
    return fruit_area,fruit_bin ,fruit_final,skin_area, fruit_contour, pix_to_cm_multiplier

**CALORIE CALCULATOR**

In [31]:
density_map = {1:0.96, 2:0.94, 3:0.72, 4:1.03, 5:0.28, 6:0.67, 7:0.52, 8:0.39, 9:0.81, 10:0.94 }
calorie_map = {1:52, 2:89, 3:151, 4:54, 5:412, 6:440, 7:158, 8:69, 9:49, 10:61}
skin_multiplier = 5*2.3

In [32]:
def computerCalories(label, volume): 
	calorie = calorie_map[int(label)]
	density = density_map[int(label)]
	mass = volume*density*1.0
	total_calories = (calorie/100.0)*mass
	return mass, total_calories, calorie

In [33]:
def computeVolume(label, area, skin_area, pix_to_cm_multiplier, fruit):
	fruit_area = (area/skin_area)*skin_multiplier 
	label = int(label)
	volume = 100
	if label == 2: 
		fruit_rect = cv2.minAreaRect(fruit)
		height = max(fruit_rect[1])*pix_to_cm_multiplier
		radius = fruit_area/(2.0*height)
		volume = np.pi*radius*radius*height
	else:
		radius = np.sqrt(fruit_area/np.pi)
		volume = (4/3)*np.pi*radius*radius*radius

	return volume

In [34]:
def calories(result,img):
    img_path =img 
    fruit_areas,fruit_bin,fruit_final,skin_areas, fruits, pix_cm = calculateFoodArea(img_path)
    volume = computeVolume(result, fruit_areas, skin_areas, pix_cm, fruits)
    mass, cal, cal100 = computerCalories(result, volume)
    fruit_volumes=volume
    fruit_calories=cal
    fruit_calories_100grams=cal100
    fruit_mass=mass
    return fruit_calories

**RUN**

In [35]:
from flask import Flask,render_template,flash, request, redirect, url_for
from werkzeug.utils import secure_filename
import os
import random

In [36]:
IMG_SIZE = 400
LR = 1e-3
no_of_fruits=10

In [37]:
MODEL_NAME = ''

In [38]:
app = Flask('__name__')
UPLOAD_FOLDER = 'static\images'
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg', 'gif'])

In [39]:
def allowed_file(filename):
	return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

In [40]:
@app.route('/predict', methods=['POST'])
def upload_image():
    if 'file' not in request.files:
       flash('No file part')
       return redirect(request.url)
    file = request.files['file']
    if file.filename == '':
        flash('No image selected for uploading')
        return redirect(request.url)
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        image_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(image_path)
        model_save_at=os.path.join("model",MODEL_NAME)
        model = keras.models.load_model(model_save_at)
        model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
        labels=list(np.load('label.npy'))        
        img=cv2.imread(image_path)
        img1=cv2.resize(img,(IMG_SIZE,IMG_SIZE))
        img1 = np.array(img1).reshape(1,400,400,3)
        model_out=model.predict(img1)
        result=np.argmax(model_out)
        name = labels[result].lower()
        cal=str(round(calories(result+1,img),2))
        if name == "apple":
           cal = "Calories: "+cal + " KCal"
           food_name = "Apple"
           desc = "Apples are one of the most popular fruits — and for good reason.\n They’re an exceptionally healthy fruit with many research-backed benefits.\n Apples May Be Good for Weight Loss and your heart. They May Have Prebiotic Effects and Promote Good Gut Bacteria.\n"
        elif name == "banana":
           cal = "Calories: "+cal + " KCal"
           food_name = "Banana"
           desc = "Bananas are one of the most popular fruits worldwide.\n They contain essential nutrients fiber, potassium, folate, and antioxidants, such as vitamin C that can have a protective impact on health.\nBananas are rich in soluble fiber.\n During digestion, soluble fiber dissolves in liquid to form a gel. It’s also what gives bananas their sponge-like texture.\n"
        elif name == "beans":
           cal = "Calories: "+cal + " KCal"
           food_name = "Beans"
           desc = "Green beans are full of fiber, which is an important nutrient for many reasons.\n Soluble fiber, in particular, may help to improve the health of your heart by lowering your LDL cholesterol (bad cholesterol) levels.\n Green beans, string beans, or snap beans are a rich source of vitamins A, C, and K, and of folic acid and fiber.\n"
        elif name == "egg":
           cal = "Calories: "+cal + " KCal"
           food_name = "Egg"
           desc = "Eggs have lots of nutrients, but the health boost you get depends on the way you prepare the meal.\n Hard-boiled eggs are an excellent source of lean protein.\n They are also a source of vitamin A, vitamin D, calcium, and iron. \n"
        elif name == "doughnut":
           cal = "Calories: "+cal + " KCal"
           food_name = "Doughnuts"
           desc = "Doughnuts are usually deep fried from a flour dough, but other types of batters can also be used. Various toppings and flavorings are used for different types, such as sugar, chocolate or maple glazing.\n Doughnuts may also include water, leavening, eggs, milk, sugar, oil, shortening, and natural or artificial flavors.\n The two most common types are the ring doughnut and the filled doughnut, which is injected with fruit preserves (the jelly doughnut), cream, custard, or other sweet fillings.\n"
        elif name == "mooncake":
           cal = "Calories: "+cal + " KCal"
           food_name = "Moon Cake"
           desc = "Mooncakes are round or square in shape, and have a pastry exterior with a dense, sweet filling usually made from lotus paste.\n Many of them also contain a salted duck yolk. Mooncakes are often sliced into wedges, and shared with the family along with tea.\n Mooncakes are composed of two parts, the skin and the filling.\ The skin is made with flour, golden syrup, alkaline water, and cooking oil. The traditional lotus paste filling is made with lotus seeds, with the option to include sesame and nuts.\n"
        elif name == "pasta":
           cal = "Calories: "+cal + " KCal"
           food_name = "Spaghetti Pasta"
           desc = "Spaghetti is a long, thin, solid, cylindrical pasta. It is a staple food of traditional Italian cuisine.\n Like other pasta, spaghetti is made of milled wheat and water and sometimes enriched with vitamins and minerals.\n Spaghetti is made from ground grain (flour) and water. Whole-wheat and multigrain spaghetti are also available.\nRegular spaghetti is fairly neutral, diet-wise, but whole-wheat spaghetti can be a good source of fiber.\n"
        elif name == "grape":
           cal = "Calories: "+cal + " KCal"
           food_name = "Grapes"
           desc = "Grapes offer a wealth of health benefits due to their high nutrient and antioxidant contents.\nCompounds found in grapes may help protect against high cholesterol levels by decreasing cholesterol absorption. \n"
        elif name == "orange":
           cal = "Calories: "+cal+ " KCal"
           food_name = "Orange"
           desc = "Oranges are a treasure trove of nutrients and protective plant compounds, including vitamins, minerals, and antioxidants. \n Oranges are a good source of fiber and a rich source of vitamin C and folate, among many other beneficial nutrients.\n Oranges may benefit heart health, reduce the risk of some chronic diseases, enhance iron absorption, and support a healthy immune response.\n"
        elif name == "qiwi":
           cal = "Calories: "+cal + " KCal"
           food_name = "Qiwi"
           desc = "Qiwis are high in Vitamin C  and dietary fiber and provide a variety of health benefits.\n This tart fruit can support heart health, digestive health, and immunity. \n The kiwi is a healthy choice of fruit and is rich with vitamins and antioxidants.\n Its tart flavor, pleasing texture, and low calorie count make it a delicious and healthy option for snacking, sides, or a unique dessert. \nThe kiwifruit possesses properties that lower blood pressure. By helping to maintain a healthy blood pressure and providing a boost of Vitamin C, the kiwifruit can reduce the risk of stroke  and heart disease.\n"
        else:
            cal = "Calories: "+cal + " KCal"
            food_name = "Pasta"
            desc = "Spaghetti is a long, thin, solid, cylindrical pasta. It is a staple food of traditional Italian cuisine.\n Like other pasta, spaghetti is made of milled wheat and water and sometimes enriched with vitamins and minerals.\n Spaghetti is made from ground grain (flour) and water. Whole-wheat and multigrain spaghetti are also available.\nRegular spaghetti is fairly neutral, diet-wise, but whole-wheat spaghetti can be a good source of fiber.\n"
        return render_template('predict.html', filename=filename,name=food_name,calories=cal,description=desc)
    else:
        flash('Allowed image types are -> png, jpg, jpeg, gif')
        return redirect(request.url)   

In [41]:
@app.route('/display/<filename>')
def display_image(filename):	
	return redirect(url_for('static', filename='images/' + filename), code=301)

In [42]:
@app.route('/')
def main_page():
    return render_template('index.html')

In [43]:
@app.route('/predict')
def predict():
    return render_template('predict.html')

In [44]:
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__name__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

d:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
